In [1]:
import numpy as np , pandas as pd
import pyodbc
import statistics
from tqdm import tqdm
import time
import math
import datetime
from datetime import datetime
from configparser import ConfigParser
import pyodbc

In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [4]:
cursor.execute('''
WITH CTE AS(
SELECT REPORT_ID,
CASE WHEN TRANSACTION_RISK+ENTITY_RISK+GOS_RISK = 0 THEN 0
ELSE
ROUND((SQUARE(TRANSACTION_RISK)+SQUARE(ENTITY_RISK)+SQUARE(GOS_RISK))/(TRANSACTION_RISK+ENTITY_RISK+GOS_RISK),0) END AS RISK_SCORE_WEIGHTAGE
FROM (
SELECT REPORT_ID,AVG(TRANSACTION_RISK) AS TRANSACTION_RISK,
AVG(ENTITY_RISK) AS ENTITY_RISK,
AVG(GOS_RISK) AS GOS_RISK
FROM (
SELECT DISTINCT RTSS.REPORT_ID,
COALESCE(RT.RISK_SCORE,0) AS TRANSACTION_RISK,
ENTITY_ID,ENTITY_TYPE,
COALESCE(RP.RISK_SCORE,RO.RISK_SCORE,0) AS ENTITY_RISK,
COALESCE(RG.RISK_SCORE,0) AS GOS_RISK
FROM FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG RTSS WITH (NOLOCK)
LEFT JOIN FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION RT WITH (NOLOCK) ON RT.REPORT_ID = RTSS.REPORT_ID
LEFT JOIN FINCORE_DB.Fincore_Analytics.RISK_PERSON RP WITH (NOLOCK) ON RTSS.ENTITY_ID = RP.PERSON_MASTER_ID AND RTSS.ENTITY_TYPE = 'I' 
LEFT JOIN FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION RO WITH (NOLOCK) ON RTSS.ENTITY_ID = RO.ORGANIZATION_MASTER_ID AND RTSS.ENTITY_TYPE = 'O' 
LEFT  JOIN FINCORE_DB.Fincore_Analytics.RISK_GOS RG WITH (NOLOCK) ON RG.REPORT_ID = RTSS.REPORT_ID
)T
GROUP BY REPORT_ID)T
WHERE REPORT_ID NOT IN (
select REPORT_ID from Fincore_Analytics.RISK_REPORT_RULES A
inner join Fincore_Analytics.RISK_LKP_RULES B 
on A.RULE_NO = B.RULE_NO
where B.PERSON_TYPE = 'Report' and B.RISK_TYPE = 'Auto High Rules ')
)
UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT
SET RISK_SCORE_WEIGHTED_AVG = CTE.RISK_SCORE_WEIGHTAGE
FROM CTE 
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_REPORT RR
ON CTE.REPORT_ID = RR.REPORT_ID
WHERE CTE.REPORT_ID NOT IN (
select REPORT_ID from Fincore_Analytics.RISK_REPORT_RULES A
inner join Fincore_Analytics.RISK_LKP_RULES B 
on A.RULE_NO = B.RULE_NO
where B.PERSON_TYPE = 'Report' and B.RISK_TYPE = 'Auto High Rules ')
''')
conn.commit()


In [5]:
cursor.execute('''
WITH CTE AS(
SELECT 
REPORT_ID,
CASE WHEN (RISK_SCORE_QUERY+RISK_SCORE_WEIGHTAGE) = 0 THEN 0
ELSE ROUND(((SQUARE(RISK_SCORE_QUERY)+SQUARE(RISK_SCORE_WEIGHTAGE))/(RISK_SCORE_QUERY+RISK_SCORE_WEIGHTAGE)),0) END AS RISK_SCORE
FROM (
SELECT REPORT_ID,
COALESCE(RISK_SCORE_QUERY,0) AS RISK_SCORE_QUERY,
COALESCE(RISK_SCORE_WEIGHTED_AVG,0) AS RISK_SCORE_WEIGHTAGE
FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT RR WITH (NOLOCK)
)T
WHERE T.REPORT_ID NOT IN (
select REPORT_ID from Fincore_Analytics.RISK_REPORT_RULES A
inner join Fincore_Analytics.RISK_LKP_RULES B 
on A.RULE_NO = B.RULE_NO
where B.PERSON_TYPE = 'Report' and B.RISK_TYPE = 'Auto High Rules ')
)
UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT
SET RISK_SCORE = CTE.RISK_SCORE
FROM CTE 
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_REPORT RR
ON CTE.REPORT_ID = RR.REPORT_ID
''')
conn.commit()

In [ ]:
cursor.execute('''UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT SET RISK_SCORE = 1 WHERE RISK_SCORE = 0''')
conn.commit()

In [7]:
store -r no_of_records

no stored variable or alias no_of_records


In [8]:
store -r job_id

no stored variable or alias job_id


In [9]:
cursor.execute('''UPDATE FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN SET JOB_END_DATE = GETDATE() , JOB_STATUS = 'Success', no_of_record = {}  where job_run_id = {} '''.format(no_of_records,job_id))
conn.commit()

NameError: name 'no_of_records' is not defined

In [ ]:
conn.close()